In [103]:
using JuMP, CPLEX, Plots, Printf

In [104]:
mpy_carbon = 31000 ; # Maximum Production per year (GWh)
mpy_oil    = 15000 ;
mpy_gas    = 22000 ;
mpy_res    = 10000 ;

cost_carbon = 30 ;  # Production cost ($/Mwh)
cost_oil    = 75 ;
cost_gas    = 60 ;
cost_res    = 90 ;

co2_carbon  = 1.44 ; # CO2 emission coefficient (t/MWh)
co2_oil     = 0.72 ;
co2_gas     = 0.45 ;
co2_res     = 0    ;

y_demand    = 64000 ; # Yearly energy demand (GWh)

base_load   = 0.6 ; # Enery load per production period
medium_load = 0.3 ;
peak_load   = 0.1 ;






In [105]:
power_mo = Model(CPLEX.Optimizer);

In [106]:
@variable(power_mo, carbon_base >= 0, start = base_load*y_demand);


In [107]:
@variable(power_mo, carbon_medium >= 0, start = medium_load*y_demand);

In [108]:
@variable(power_mo, oil_medium >= 0, start = medium_load*y_demand);

In [109]:
@variable(power_mo, oil_peak >=0,  start = medium_load*y_demand);

In [110]:
@variable(power_mo, gas_base >= 0, start = base_load*y_demand);

In [111]:
@variable(power_mo, gas_medium >= 0, start = medium_load*y_demand);

In [112]:
@variable(power_mo, gas_peak>= 0, start = peak_load*y_demand);

In [113]:
@variable(power_mo, res_base >= 0, start = base_load*y_demand);

In [114]:
@variable(power_mo, res_peak >= 0, start = peak_load*y_demand);

In [115]:
@variable(power_mo, carbon >= 0, start = 0.5*mpy_carbon);

In [116]:
@variable(power_mo, oil >= 0, start = 0.5*mpy_oil);

In [117]:
@variable(power_mo, gas >= 0, start = 0.5*mpy_gas);

In [118]:
@variable(power_mo, res >= 0, start = 0.5*mpy_res);

In [119]:
@variable(power_mo, co2 >= 0, start = 50000);

In [120]:
@variable(power_mo, cost >=0, start = 1e06);

In [121]:
@constraint(power_mo, c1, co2 == co2_carbon*carbon + co2_oil*oil + co2_gas*gas + co2_res*res );

In [122]:
@constraint(power_mo, c2,  carbon == carbon_base +carbon_medium);

In [123]:
@constraint(power_mo, c3, oil == oil_medium + oil_peak) ;

In [124]:
@constraint(power_mo, c4, gas == gas_base + gas_medium + gas_peak); 

In [125]:
@constraint(power_mo, c5, res == res_base + res_peak) ;

In [126]:
@constraint(power_mo, c6, carbon_base + gas_base + res_base >= base_load*y_demand) ;

In [127]:
@constraint(power_mo, c7, carbon_medium + oil_medium + gas_medium >= medium_load*y_demand);

In [128]:
@constraint(power_mo, c8, oil_peak + gas_peak + res_peak >= peak_load*y_demand) ;

In [129]:
@constraint(power_mo, c9, carbon <= mpy_carbon);

In [130]:
@constraint(power_mo, c10, oil <= mpy_oil);

In [131]:
@constraint(power_mo, c11, gas <= mpy_gas);

In [132]:
@constraint(power_mo, c12, res <= mpy_res);

In [133]:
@constraint(power_mo, c13, cost == cost_carbon*carbon + cost_oil*oil + cost_gas*gas + cost_res*res)

c13 : cost - 30 carbon - 75 oil - 60 gas - 90 res = 0.0

In [134]:
@objective(power_mo, Min, cost)

cost

In [135]:
optimize!(power_mo)

Tried aggregator 1 time.
LP Presolve eliminated 6 rows and 2 columns.
Reduced LP has 7 rows, 13 columns, and 22 nonzeros.
Presolve time = 0.00 sec. (0.01 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             0.000000


In [136]:
println("Objective function value = ", JuMP.objective_value(power_mo))
println("CO2 Emissions = ", JuMP.value(co2))

println("-----------------------------\n")
println("    GLOBAL RESULTS\n")
println("Carbon = ", JuMP.value(carbon))
println("Oil    = ", JuMP.value(oil))
println("Gas    = ", JuMP.value(gas))
println("RES    = ", JuMP.value(res))
println("-----------------------------\n")
println("    BASE PERIOD\n")
println("Carbon = ", JuMP.value(carbon_base))
println("Gas    = ", JuMP.value(gas_base))
println("RES    = ", JuMP.value(res_base))
println("-----------------------------\n")

println("    MEDIUM PERIOD\n")
println("Carbon = ", JuMP.value(carbon_medium))
println("OiL    = ", JuMP.value(oil_medium))
println("gas    = ", JuMP.value(gas_medium))
println("-----------------------------\n")

println("    PEAK PERIOD\n")
println("OiL    = ", JuMP.value(oil_peak))
println("gas    = ", JuMP.value(gas_peak))
println("RES    = ", JuMP.value(res_peak))
println("-----------------------------\n")
   


Objective function value = 3.075e6
CO2 Emissions = 62460.0
-----------------------------

    GLOBAL RESULTS

Carbon = 31000.0
Oil    = 11000.0
Gas    = 22000.0
RES    = 0.0
-----------------------------

    BASE PERIOD

Carbon = 22800.0
Gas    = 15600.0
RES    = 0.0
-----------------------------

    MEDIUM PERIOD

Carbon = 8200.0
OiL    = 11000.0
gas    = 0.0
-----------------------------

    PEAK PERIOD

OiL    = 0.0
gas    = 6400.0
RES    = 0.0
-----------------------------

